# Assignment 1 – Backprop

In [1]:
#@title Library Imports [do not change]

import importlib
!git clone https://www.github.com/rycolab/intro-nlp-f23.git
utils = importlib.import_module("intro-nlp-f23.assignment_1.utils")

import re
import random
from collections import defaultdict
import itertools
from abc import ABC, abstractmethod
import math

Cloning into 'intro-nlp-f23'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 12 (delta 1), reused 12 (delta 1), pack-reused 0
Receiving objects: 100% (12/12), 4.25 KiB | 4.25 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [2]:
#@title Select and Parse Math Problems [do not change]

#@markdown select math problem

math_problem_i = "3" #@param [0,1,2,3]
math_problem = utils.MATH_PROBLEMS[int(math_problem_i)]
print(math_problem)

parser = utils.Parser()
infix, in_vars = parser.parse(math_problem["problem"], in_vars = math_problem["in_vars"])
print(infix, in_vars)

{'problem': 'z + sin(x^(2) + (y * exp(z)))', 'in_vars': {'x': 2.0, 'y': -1.0, 'z': 4.0}, 'output': 3.67, 'derivative': {'x': 3.78, 'y': 51.6, 'z': -50.6}}
['z', '+', ['sin', [['x', '^', 2], '+', ['y', '*', ['exp', 'z']]]]] {'x': 2.0, 'z': 4.0, 'y': -1.0}


In [3]:
from collections.abc import ItemsView
#@title ToDo1: Building


class Builder():

    def __init__(self, infix: list, in_vars: dict = {}):
        """
        infix: list of infix notation parse, e.g. [['exp', 2], '-', 3]
        in_vars: dict of input variables to ensure they are not used as intermediate or output variables
        RETURN: computation graph in a data structure of your choosing
        """

        ## some alphabetical vars to use as intermediate and output variables minus the input vars to avoid duplicates
        avail_vars = list(map(chr, range(97, 123))) + list(map(chr, range(945, 969)))
        if len(in_vars.keys()) > 0:
            avail_vars = set(avail_vars) - set(in_vars)
        self.avail_vars = sorted(list(set(avail_vars)), reverse=True)

        self.infix = infix
        self.graph = in_vars.copy()

        ## ToDO: implement and set self.graph

        self.avail_counter = 0
        self.build_computational_graph()

    def computational_graph(self, item, c_node):
        operands = []
        node = c_node

        if isinstance(item, (str, int)):
            pass

        elif len(item) == 2:
            operator = item[0]
            if isinstance(item[1], (str, int)):
                operands += [item[1]]
            else:
                self.avail_counter += 1
                operands += [self.avail_vars[self.avail_counter]]
            self.graph[node] = (operator, operands)
            self.computational_graph(item[1], operands[0])

        elif len(item) == 3:
            operator = item[1]
            if isinstance(item[0], (str, int)):
                operands += [item[0]]
            else:
                self.avail_counter += 1
                operands += [self.avail_vars[self.avail_counter]]
            #if isinstance(item[2], int) or isinstance(item[2], str):
            if isinstance(item[2], (str, int)):
                operands += [item[2]]
            else:
                self.avail_counter += 1
                operands += [self.avail_vars[self.avail_counter]]
            self.graph[node] = (operator, operands)
            self.computational_graph(item[0], operands[0])
            self.computational_graph(item[2], operands[1])


    def build_computational_graph(self):
        self.computational_graph(self.infix, self.avail_vars[0])


if __name__ == '__main__':

    g = Builder(infix, in_vars)
    print(g.graph)

{'x': 2.0, 'z': 4.0, 'y': -1.0, 'ψ': ('+', ['z', 'χ']), 'χ': ('sin', ['φ']), 'φ': ('+', ['υ', 'τ']), 'υ': ('^', ['x', 2]), 'τ': ('*', ['y', 'σ']), 'σ': ('exp', ['z'])}


In [4]:
#@title ToDo2: Operations

class Operator(ABC):

    @abstractmethod
    def f(self, a, b = None) -> float:
        raise NotImplementedError()
        return f_res

    @abstractmethod
    def df(self, a, b = None) -> list:
        raise NotImplementedError()
        return [df_res]

class Exp(Operator):

    def f(self, a, b = None):
        return math.exp(a)

    def df(self, a, b = None):
        return [math.exp(a)]

class Log(Operator):
    ## natural logarithm

    def f(self, a, b = None):
        ## ToDO: implement
        return math.log(a)

    def df(self, a, b = None):
        ## ToDO: implement
        return [1 / a]

class Mult(Operator):

    def f(self, a, b):
        return a * b

    def df(self, a, b=None):
        return [b, a]

class Div(Operator):

    def f(self, a, b):
        ## ToDO: implement
        return a / b

    def df(self, a, b):
        ## ToDO: implement
        return [1 / b, -a / (b**2)]

class Add(Operator):

    def f(self, a, b):
        ## ToDO: implement
        return a + b

    def df(self, a, b = None):
        ## ToDO: implement
        return [1, 1]

class Sub(Operator):

    def f(self, a, b = None):
        ## ToDO: implement
        return a - b

    def df(self, a, b = None):
        ## ToDO: implement
        return [1, -1]

class Pow(Operator):

    def f(self, a, b):
        return a**b

    def df(self, a, b):
        if a <= 0: ## work-around: treat as unary operation if -a^b
            return [b * (a ** (b - 1))]
        else:
            return [b * (a ** (b - 1)), (a ** b) * math.log(a)]

class Sin(Operator):

    def f(self, a, b=None):
        ## ToDO: implement
        return math.sin(a)

    def df(self, a, b=None):
        ## ToDO: implement
        return [math.cos(a)]

class Cos(Operator):

    def f(self, a, b=None):
        ## ToDO: implement
        return math.cos(a)

    def df(self, a, b=None):
        ## ToDO: implement
        return [- math.sin(a)]

In [5]:
#@title ToDo 3: Executing

class Executor():

    def __init__(self, graph: dict, in_vars: dict = {}):
        """
        graph: computation graph in a data structure of your choosing
        in_vars: dict of input variables, e.g. {"x": 2.0, "y": -1.0}
        """
        self.graph = graph
        self.in_vars = in_vars
        self.fn_map = {"log": Log(), "exp": Exp(), "+": Add(), "-": Sub(), "^": Pow(), "sin": Sin(), "cos": Cos(), "*": Mult(), "/": Div()}
        self.output = -1
        self.derivative = {}
        self.intermediate_values = {}
        self.intermediate_derivatives = {}
        self.prod = 1
        for i in self.in_vars:
            self.derivative[i] = 0

    ## forward execution____________________________

    def forward(self, ):
        ## ToDO: implement and set self.output

        for key, value in sorted(self.graph.items()):

            if key in self.in_vars:
                self.intermediate_values[key] = value

            else:
                operation = self.fn_map[value[0]]
                operands = []
                for item in value[1]:
                    if item in self.intermediate_values:
                        operands.append(self.intermediate_values[item])
                    else:
                        operands.append(item)
                self.intermediate_values[key] = operation.f(*operands)
                self.intermediate_derivatives[key] = operation.df(*operands)

        self.output = self.intermediate_values[sorted(self.graph.keys())[-1]]

    ## backward execution____________________________

    def build_derivative(self, key, value):
        operator, operands = value

        if len(operands) == 1:
            if operands[0] in self.in_vars:
                self.prod *= self.intermediate_derivatives[key][0]
                self.derivative[operands[0]] += self.prod
                self.prod /= self.intermediate_derivatives[key][0]
            elif not isinstance(operands[0], int):
                self.prod *= self.intermediate_derivatives[key][0]
                self.build_derivative(operands[0], self.graph[operands[0]])
                self.prod /= self.intermediate_derivatives[key][0]

        elif len(operands) == 2:
            for i, operand in enumerate(operands):
                if operand in self.in_vars:
                    self.prod *= self.intermediate_derivatives[key][i]
                    self.derivative[operand] += self.prod
                    self.prod /= self.intermediate_derivatives[key][i]
                elif not isinstance(operand, int):
                    self.prod *= self.intermediate_derivatives[key][i]
                    self.build_derivative(operand, self.graph[operand])
                    self.prod /= self.intermediate_derivatives[key][i]

    def backward(self, ):
        ## ToDO: implement and set self.derivative
        #self.derivative = {}
        self.key, self.value = sorted(self.graph.items(), reverse=True)[0]
        self.build_derivative(self.key, self.value)


if __name__ == '__main__':
  e = Executor(g.graph, in_vars=in_vars)
  e.forward()
  e.backward()
  print(e.output)
  print(e.derivative)

3.673434487962201
{'x': 3.7806982769800404, 'z': -50.604782939151534, 'y': 51.604782939151534}


In [6]:
#@title Test Function for Debugging [do not change]

utils.test_backprop(Builder, Executor, math_problem)


0: problem: z + sin(x^(2) + (y * exp(z))), in_vars: {'x': 2.0, 'y': -1.0, 'z': 4.0}
SUCCESS output: 3.67
SUCCESS derivative: {'x': 3.78, 'z': -50.6, 'y': 51.6}


In [7]:
#@title Test Function for Grading [do not change]

utils.test_backprop(Builder, Executor)


0: problem: x/y, in_vars: {'x': 2.0, 'y': 4.0}
SUCCESS output: 0.5
SUCCESS derivative: {'x': 0.25, 'y': -0.12}

1: problem: exp(x) - (y * 4), in_vars: {'x': 3.0, 'y': 3.0}
SUCCESS output: 8.09
SUCCESS derivative: {'x': 20.09, 'y': -4.0}

2: problem: (x^2 - 1) * (y+3), in_vars: {'x': 4.0, 'y': 2.0}
SUCCESS output: 75.0
SUCCESS derivative: {'x': 40.0, 'y': 15.0}

3: problem: z + sin(x^(2) + (y * exp(z))), in_vars: {'x': 2.0, 'y': -1.0, 'z': 4.0}
SUCCESS output: 3.67
SUCCESS derivative: {'x': 3.78, 'z': -50.6, 'y': 51.6}
